# Prepare *Enviroment*

In [0]:
#all models trained using tensorflow 1.15.2
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.15.2
!pip install numpy==1.17.4

Uninstalling tensorflow-2.2.0rc4:
  Successfully uninstalled tensorflow-2.2.0rc4
     |████████████████████████████████| 411.0MB 30kB/s 
     |████████████████████████████████| 512kB 2.7MB/s 
     |████████████████████████████████| 3.8MB 8.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=f547e7daf0ecf4a8545d6f51f97cd94f0a6baf25679e204e9fb49320c7b23d3a
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1

In [0]:
#intstalling object detection API
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2590, done.
remote: Counting objects: 100% (2590/2590), done.
remote: Compressing objects: 100% (2263/2263), done.
remote: Total 2590 (delta 499), reused 1347 (delta 290), pack-reused 0
Receiving objects: 100% (2590/2590), 124.11 MiB | 15.63 MiB/s, done.
Resolving deltas: 100% (499/499), done.
Checking out files: 100% (2495/2495), done.


In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [0]:
%%bash 
cd models/research
pip install .


Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1017525 sha256=73c92638ed68e6b1a6a9af87d9101995729e4d521fc01c745c4028a0f80e71cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-b6lfxbvy/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection


# Imports

In [0]:
import numpy as np
import tensorflow as tf
from PIL import Image
import cv2
from matplotlib import pyplot as plt
%matplotlib inline


In [0]:
%matplotlib inline

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import shutil
shutil.copy('//content/drive/My Drive/MIU/Datasets/PlantDOC_Modified.zip', '/content/')
shutil.copy('/content/drive/My Drive/disease.zip', '/content/')

'/content/disease.zip'

In [0]:
!unzip PlantDOC_Modified.zip
!unzip disease.zip

Streaming output truncated to the last 5000 lines.
  inflating: Train/anntos/img0164.xml  
  inflating: Train/anntos/img0165.xml  
  inflating: Train/anntos/img0166.xml  
  inflating: Train/anntos/img0167.xml  
  inflating: Train/anntos/img0168.xml  
  inflating: Train/anntos/img0169.xml  
  inflating: Train/anntos/img0170.xml  
  inflating: Train/anntos/img0171.xml  
  inflating: Train/anntos/img0172.xml  
  inflating: Train/anntos/img0173.xml  
  inflating: Train/anntos/img0174.xml  
  inflating: Train/anntos/img0175.xml  
  inflating: Train/anntos/img0176.xml  
  inflating: Train/anntos/img0177.xml  
  inflating: Train/anntos/img0178.xml  
  inflating: Train/anntos/img0179.xml  
  inflating: Train/anntos/img0180.xml  
  inflating: Train/anntos/img0181.xml  
  inflating: Train/anntos/img0182.xml  
  inflating: Train/anntos/img0183.xml  
  inflating: Train/anntos/img0184.xml  
  inflating: Train/anntos/img0185.xml  
  inflating: Train/anntos/img0186.xml  
  inflating: Train/anntos/img

In [0]:
MODEL_NAME = 'inference_graph'
PATH_TO_FROZEN_GRAPH = "/content/content/models/research/object_detection/inference_graph/frozen_inference_graph.pb"
PATH_TO_LABELS = '/content/labels_map.pbtxt'

## Load a (frozen) Tensorflow model into memory.

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map


In [0]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [0]:
from imutils import paths
TEST_IMAGE_PATHS = list(paths.list_images("/content/tr"))

In [0]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [0]:
#visualize prediction using matplotlib
def show_inference(TEST_IMAGE_PATHS) :
  for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  image_np = load_image_into_numpy_array(image)
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)

In [0]:
my_list=["img"+'{0:04}'.format(num)+".jpg" for num in range(0, 10000)]


In [0]:
import os
os.mkdir("/content/Results")


In [0]:
cd /content/Results

/content/Results


In [0]:
#Save_predictions
import cv2
i=0
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  image_np = load_image_into_numpy_array(image)
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  im_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
  cv2.imwrite(my_list[i], im_bgr)
  print(my_list[i]+" generated")
  i+=1

test_10000.jpg generated
test_10001.jpg generated
test_10002.jpg generated
test_10003.jpg generated
test_10004.jpg generated
test_10005.jpg generated
test_10006.jpg generated
test_10007.jpg generated
test_10008.jpg generated
test_10009.jpg generated


In [0]:
!zip -r /content/results.zip /content/Results

  adding: content/Results/ (stored 0%)
  adding: content/Results/test_10026.jpg (deflated 6%)
  adding: content/Results/test_10037.jpg (deflated 4%)
  adding: content/Results/test_10004.jpg (deflated 3%)
  adding: content/Results/test_10034.jpg (deflated 4%)
  adding: content/Results/test_10044.jpg (deflated 6%)
  adding: content/Results/test_10002.jpg (deflated 0%)
  adding: content/Results/test_10041.jpg (deflated 3%)
  adding: content/Results/test_10014.jpg (deflated 3%)
  adding: content/Results/test_10048.jpg (deflated 4%)
  adding: content/Results/test_10035.jpg (deflated 1%)
  adding: content/Results/test_10042.jpg (deflated 2%)
  adding: content/Results/test_10009.jpg (deflated 0%)
  adding: content/Results/test_10025.jpg (deflated 2%)
  adding: content/Results/test_10007.jpg (deflated 16%)
  adding: content/Results/test_10030.jpg (deflated 1%)
  adding: content/Results/test_10005.jpg (deflated 5%)
  adding: content/Results/test_10019.jpg (deflated 3%)
  adding: content/Results

In [0]:
def crop (image_path):
  leaves_bbox=[]
  image = Image.open(image_path)
  image_np = load_image_into_numpy_array(image)
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  (im_width, im_height) = image.size
  n_true_detects = len([i for i in output_dict['detection_scores'] if i > 0.5]) 
  for i in range(n_true_detects):
    bbox=output_dict['detection_boxes'][i]
    ymin=bbox[0]*im_height
    xmin=bbox[1]*im_width
    ymax=bbox[2]*im_height
    xmax=bbox[3]*im_width
    im = Image.open(image_path)
    im = im.crop((xmin, ymin, xmax, ymax))
    path="/content/croped_leaves/"+"leaf"+str(i)+".jpg"
    im.save(path)
  






In [0]:
crop ("/content/test/image1.jpg")